# Load and preprocess data

Load the CIFAR-10 dataset and preprocess the data by scaling the pixel values to be between 0 and 1 and converting the class labels to one-hot encoded vectors

In [6]:
from keras.datasets import cifar10
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocess data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


# Define the model architecture

Define a convolutional neural network (CNN) model with four convolutional layers and two fully connected layers.

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_5 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 15, 15, 64)        18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                  

# Compile and fit the model

Compile the model with categorical cross-entropy loss and the Adam optimizer. Then, fit the model on the training data with a batch size of 64, for 10 epochs. Use the test set for validation.

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
782/782 [==============================] - 10s 9ms/step - loss: 1.3750 - accuracy: 0.5055 - val_loss: 1.0526 - val_accuracy: 0.6268
Epoch 2/10
782/782 [==============================] - 7s 9ms/step - loss: 0.9237 - accuracy: 0.6758 - val_loss: 0.8686 - val_accuracy: 0.6945
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.7288 - accuracy: 0.7452 - val_loss: 0.7542 - val_accuracy: 0.7379
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.5751 - accuracy: 0.7998 - val_loss: 0.7260 - val_accuracy: 0.7516
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.4445 - accuracy: 0.8432 - val_loss: 0.7819 - val_accuracy: 0.7475
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3214 - accuracy: 0.8858 - val_loss: 0.8125 - val_accuracy: 0.7566
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2232 - accuracy: 0.9214 - val_loss: 0.9595 - val_accuracy: 0.7463
Epoch

# Extract features from the last fully connected layer

Create a feature extractor by building a new model that takes the same input as the original model but outputs the activations of the second to last layer. Use this feature extractor to transform the training and test data into feature vectors.

In [9]:
from keras.models import Model
import numpy as np
from sklearn.svm import SVC

# Extract features from last fully connected layer
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)
x_train_features = feature_extractor.predict(x_train)
x_test_features = feature_extractor.predict(x_test)


313/313 [==============================] - 1s 2ms/step


# Train an SVM classifier

Train a support vector machine (SVM) classifier with a linear kernel on the feature vectors extracted from the last fully connected layer of the CNN.

In [10]:
# Train SVM classifier
svm = SVC(kernel='linear')
svm.fit(x_train_features, np.argmax(y_train, axis=1))


SVC(kernel='linear')

# Evaluate the model

Evaluate the trained model on the test set by computing the accuracy of the SVM classifier on the extracted feature vectors of the test set.

In [11]:
# Evaluate on test set
test_acc = svm.score(x_test_features, np.argmax(y_test, axis=1))
print('Test accuracy:', test_acc)

Test accuracy: 0.7493


#  Test the Trained Model on Multiple Images
Load multiple test images, preprocess them, stack them into a single tensor, and use the trained model to predict the class labels of each image. The predicted class labels will be printed for each image.

In [21]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the test images
test_images = []
for i in range(4):
    img_path = f"/content/image{i+1}.jpeg"
    img = image.load_img(img_path, target_size=(32, 32))
    img = image.img_to_array(img)
    img = img.astype('float32') / 255
    test_images.append(img)

# Stack the input images into a single tensor
stacked_images = np.stack(test_images, axis=0)

# Predict the classes of the stacked images
class_idxs = np.argmax(model.predict(stacked_images), axis=1)

# Print the predicted class labels for each image
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i, class_idx in enumerate(class_idxs):
    print(f"Predicted class for image{i+1}: {class_labels[class_idx]}")


1/1 [==============================] - 0s 148ms/step
Predicted class for image1: horse
Predicted class for image2: airplane
Predicted class for image3: deer
Predicted class for image4: dog


# Create a requirement file

In [5]:
!pip freeze > requirements.txt
